## 導入所有必要的程式庫和宣告

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
import torchvision
torchvision.disable_beta_transforms_warning()
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import alexnet, AlexNet_Weights

from sklearn.ensemble import IsolationForest

TARGET_SIZE = (28, 28)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## 載入 MNIST 訓練資料

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize(size=TARGET_SIZE)])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

class SubDataset(Dataset):
    LABEL_TO_INDEX = {
        1: 0, 3: 1, 5: 2, 7: 3
    }
    INDEX_TO_LABEL = [
        1, 3, 5, 7
    ]

    def __init__(self, full_dataset):
        self._data_pair = list()
        for data in full_dataset:
            img, label = data
            if label in self.LABEL_TO_INDEX.keys():
                self._data_pair.append((img, label))

    def __len__(self):
        return len(self._data_pair)

    def __getitem__(self, idx):
        img, label = self._data_pair[idx]
        return img, self.LABEL_TO_INDEX[label]

class AbnormDataset(Dataset):
    NORM_LABEL = [
        1, 3, 5, 7
    ]

    def __init__(self, full_dataset):
        self._data_pair = list()
        for data in full_dataset:
            img, label = data
            self._data_pair.append((img, label not in self.NORM_LABEL))

    def __len__(self):
        return len(self._data_pair)

    def __getitem__(self, idx):
        img, abnorm = self._data_pair[idx]
        return img, abnorm

t_subset = SubDataset(trainset)
t_loader = torch.utils.data.DataLoader(t_subset, batch_size=64, shuffle=True)
v_subset = SubDataset(testset)
v_loader = torch.utils.data.DataLoader(v_subset, batch_size=64, shuffle=True)
r_dataset = AbnormDataset(trainset)
r_loader = torch.utils.data.DataLoader(r_dataset, batch_size=64, shuffle=True)
a_dataset = AbnormDataset(testset)
a_loader = torch.utils.data.DataLoader(a_dataset, batch_size=64, shuffle=True)

## 建構用於分類網路（第一小題）

In [3]:
class FullyConnect(nn.Module):
    def __init__(self, in_size,
                       out_size,
                       activation=None):
        super().__init__()
        self.act = activation
        self.linear = nn.Linear(
            in_size,
            out_size,
            bias=True
        )

    def forward(self, x):
        x = self.linear(x)
        if not self.act is None:
            x = self.act(x)
        return x

class ConvBlock(nn.Module):
    def __init__(self, in_channels,
                       out_channels,
                       kernel_size,
                       activation=None):
        super().__init__()
        self.act = activation
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            padding="same",
            bias=False,
        )
        self.bn = nn.BatchNorm2d(
            out_channels,
            eps=1e-5
        )
        nn.init.kaiming_normal_(self.conv.weight,
                                mode="fan_out",
                                nonlinearity="relu")
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if not self.act is None:
            x = self.act(x)
        return x

class ClassifyNetwork(nn.Module):
    def __init__(self):
        super(ClassifyNetwork, self).__init__()
        self.img_size = TARGET_SIZE

        self.body = nn.Sequential(
            ConvBlock(1, 32, 7, nn.SiLU()),
            ConvBlock(32, 32, 3, nn.SiLU()),
            ConvBlock(32, 2, 3, nn.SiLU())
        )

        h, w = self.img_size
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.1),
            FullyConnect(h * w * 2, 128, nn.SiLU()),
            nn.Dropout(p=0.1),
            FullyConnect(128, 4),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.body(x)
        x = self.head(x)
        return x

    def get_prob(self, x):
        x = self.forward(x)
        return self.softmax(x)

## 訓練 MNIST 分類網路（第一小題）

In [4]:
classify_net = ClassifyNetwork()
classify_net = classify_net.to(device)

cross_entroy = nn.CrossEntropyLoss()
opt = optim.SGD(classify_net.parameters(),
                lr=0.01,
                momentum=0.9,
                nesterov=True,
                weight_decay=0.001)

running_loss = list()
for e in range(10):
    classify_net.train()
    for imgs, labels in t_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        opt.zero_grad()
        loss = cross_entroy(classify_net(imgs), labels)
        loss.backward()
        opt.step()

        running_loss.append(loss.item())
        if len(running_loss) > 500:
            running_loss.pop(0)

    classify_net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in v_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = classify_net(imgs)
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
        print("epoch {} -> loss: {:.4f}, acc: {:.2f}%".format(
                  e+1, sum(running_loss)/len(running_loss), 100.0 * correct/total))

epoch 1 -> loss: 0.1040, acc: 99.19%
epoch 2 -> loss: 0.0270, acc: 99.41%
epoch 3 -> loss: 0.0183, acc: 99.29%
epoch 4 -> loss: 0.0132, acc: 99.56%
epoch 5 -> loss: 0.0118, acc: 99.41%
epoch 6 -> loss: 0.0113, acc: 99.68%
epoch 7 -> loss: 0.0097, acc: 99.66%
epoch 8 -> loss: 0.0079, acc: 99.75%
epoch 9 -> loss: 0.0089, acc: 99.58%
epoch 10 -> loss: 0.0070, acc: 99.61%


## 測試分類網路的檢測性能（第一小題）

In [5]:
classify_net.eval()
best_classify_net_acc = 0
for i in range(100):
    thres = 0.995 + 0.005 * (i/100)
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, abnorm in a_loader:
            imgs, abnorm = imgs.to(device), abnorm.to(device)
            prob, _ = torch.max(classify_net.get_prob(imgs), dim=1)
            pred_abnorm = torch.where(prob < thres, True, False)
            correct += (pred_abnorm == abnorm).sum().item()
            total += abnorm.size(0)
        classify_net_acc = correct/total
        if best_classify_net_acc < classify_net_acc:
            best_classify_net_acc = classify_net_acc
            print("Test... current best accuracy is {:.2f}%".format(100 * best_classify_net_acc))
print("Best accuracy: {:.2f}%".format(100 * best_classify_net_acc))

Test... current best accuracy is 83.75%
Test... current best accuracy is 83.76%
Test... current best accuracy is 83.77%
Test... current best accuracy is 83.79%
Test... current best accuracy is 83.84%
Test... current best accuracy is 83.88%
Test... current best accuracy is 83.92%
Test... current best accuracy is 83.97%
Test... current best accuracy is 83.99%
Test... current best accuracy is 84.04%
Test... current best accuracy is 84.09%
Test... current best accuracy is 84.13%
Test... current best accuracy is 84.15%
Test... current best accuracy is 84.18%
Test... current best accuracy is 84.24%
Test... current best accuracy is 84.28%
Test... current best accuracy is 84.29%
Test... current best accuracy is 84.33%
Test... current best accuracy is 84.39%
Test... current best accuracy is 84.44%
Test... current best accuracy is 84.50%
Test... current best accuracy is 84.51%
Test... current best accuracy is 84.55%
Test... current best accuracy is 84.59%
Test... current best accuracy is 84.64%


## 建構 AutoEncoder 的網路（第二小題）

In [6]:
class AutoEncoder(nn.Module):
    def __init__(self, ctx_size):
        super(AutoEncoder, self).__init__()
        self.img_size = TARGET_SIZE
        h, w = self.img_size

        self.encoder = nn.Sequential(
            FullyConnect(w * h, 256, nn.SiLU()),
            FullyConnect(256, 128, nn.SiLU()),
            FullyConnect(128, ctx_size, nn.SiLU()),
        )
        self.decoder = nn.Sequential(
            FullyConnect(ctx_size, 128, nn.SiLU()),
            FullyConnect(128, 256, nn.SiLU()),
            FullyConnect(256, w * h, nn.Sigmoid())
        )

    def encode(self, img):
        b, _, _, _ = img.shape
        h, w = self.img_size
        img = torch.reshape(img, (b, h * w))
        ctx = self.encoder(img)
        return ctx

    def decode(self, ctx):
        b, _ = ctx.shape
        h, w = self.img_size
        img = self.decoder(ctx)
        img = torch.reshape(img, (b, 1, h, w))
        return img

    def forward(self, x):
        ctx = self.encode(x)
        x = self.decode(ctx)
        return x

def train_auto_encoder(net, device, loader):
    net = net.to(device)
    net.train()
    bce_loss = nn.BCELoss()
    opt = optim.Adam(net.parameters(),
                     lr=0.002,
                     weight_decay=0.0)

    running_loss = list()

    for e in range(200):
        for imgs, _ in loader:
            imgs = imgs.to(device)
            opt.zero_grad()
            loss = bce_loss(net(imgs), imgs)
            loss.backward()
            opt.step()

            running_loss.append(loss.item())
            if len(running_loss) > 500:
                running_loss.pop(0)
        if (e+1) % 20 == 0:
            print("epoch {} -> loss: {:.4f}".format(
                      e+1, sum(running_loss)/len(running_loss)))
    return net

def compute_threshold(net, device, loader, factor=1.2):
    net.eval()
    bce_loss_without_reduction = nn.BCELoss(reduction='none')
    item_loss = torch.zeros(0).to(device)

    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(device)
            loss = bce_loss_without_reduction(net(imgs), imgs)
            loss = torch.flatten(loss, start_dim=1)
            loss = torch.mean(loss, dim=1)
            item_loss = torch.cat((item_loss, loss), 0)

    item_loss = item_loss.detach().cpu().numpy()
    mean = np.mean(item_loss)
    std = np.std(item_loss)
    thres = mean + factor * std
    return thres

def compute_accuracy(net, device, loader, thres):
    bce_loss_without_reduction = nn.BCELoss(reduction='none')
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, abnorm in loader:
            imgs, abnorm = imgs.to(device), abnorm.to(device)
            loss = bce_loss_without_reduction(net(imgs), imgs)
            loss = torch.flatten(loss, start_dim=1)
            loss = torch.mean(loss, dim=1)
            pred_abnorm = torch.where(loss > thres, True, False)
            correct += (pred_abnorm == abnorm).sum().item()
            total += abnorm.size(0)
    return correct/total

## 訓練 AutoEncoder 的網路（第二小題）

In [7]:
ae_net = AutoEncoder(2)
ae_net = ae_net.to(device)
ae_net = train_auto_encoder(ae_net, device, t_loader)

epoch 20 -> loss: 0.1384
epoch 40 -> loss: 0.1341
epoch 60 -> loss: 0.1315
epoch 80 -> loss: 0.1305
epoch 100 -> loss: 0.1292
epoch 120 -> loss: 0.1281
epoch 140 -> loss: 0.1274
epoch 160 -> loss: 0.1268
epoch 180 -> loss: 0.1259
epoch 200 -> loss: 0.1257


## 測試 AutoEncoder 的檢測性能（第二小題）

In [8]:
best_ae_net_acc = 0
for i in range(30):
    factor = i/20
    thres = compute_threshold(ae_net, device, t_loader, factor)
    ae_net_acc = compute_accuracy(ae_net, device, a_loader, thres)
    if best_ae_net_acc > ae_net_acc:
        break
    best_ae_net_acc = ae_net_acc
    print("Test... current best accuracy is {:.2f}%".format(100 * best_ae_net_acc))
print("Best accuracy: {:.2f}%".format(100 * best_ae_net_acc))

Test... current best accuracy is 77.27%
Test... current best accuracy is 77.83%
Test... current best accuracy is 78.37%
Test... current best accuracy is 78.83%
Test... current best accuracy is 79.41%
Test... current best accuracy is 80.15%
Test... current best accuracy is 80.72%
Test... current best accuracy is 81.28%
Test... current best accuracy is 82.01%
Test... current best accuracy is 82.59%
Test... current best accuracy is 83.25%
Test... current best accuracy is 83.75%
Test... current best accuracy is 84.42%
Test... current best accuracy is 84.97%
Test... current best accuracy is 85.41%
Test... current best accuracy is 85.91%
Test... current best accuracy is 86.18%
Test... current best accuracy is 86.65%
Test... current best accuracy is 87.02%
Test... current best accuracy is 87.28%
Test... current best accuracy is 87.47%
Test... current best accuracy is 87.68%
Test... current best accuracy is 87.89%
Test... current best accuracy is 88.01%
Test... current best accuracy is 88.04%


## 建構 Denoising AutoEncoder 的網路（第三小題）

In [9]:
class DenoisingAutoEncoder(AutoEncoder):
    def __init__(self, ctx_size):
        super(DenoisingAutoEncoder, self).__init__(ctx_size)

    def add_noise(self, img, noise_factor=0.25):
        noise = torch.randn_like(img) * noise_factor
        noisy_img = img + noise
        return torch.clamp(noisy_img, 0., 1.)

    def forward(self, x):
        x = self.add_noise(x)
        ctx = self.encode(x)
        x = self.decode(ctx)
        return x

## 訓練 Denoising AutoEncoder 的網路（第三小題）

In [10]:
dae_net = DenoisingAutoEncoder(2)
dae_net = dae_net.to(device)
dae_net = train_auto_encoder(dae_net, device, t_loader)

epoch 20 -> loss: 0.1415
epoch 40 -> loss: 0.1386
epoch 60 -> loss: 0.1369
epoch 80 -> loss: 0.1361
epoch 100 -> loss: 0.1354
epoch 120 -> loss: 0.1350
epoch 140 -> loss: 0.1343
epoch 160 -> loss: 0.1339
epoch 180 -> loss: 0.1335
epoch 200 -> loss: 0.1330


## 測試 Denoising AutoEncoder 的檢測性能（第三小題）

In [11]:
best_dae_net_acc = 0
for i in range(30):
    factor = i/20
    thres = compute_threshold(dae_net, device, t_loader, factor)
    dae_net_acc = compute_accuracy(dae_net, device, a_loader, thres)
    if best_dae_net_acc > dae_net_acc:
        break
    best_dae_net_acc = dae_net_acc
    print("Test... current best accuracy is {:.2f}%".format(100 * best_dae_net_acc))
print("Best accuracy: {:.2f}%".format(100 * best_dae_net_acc))

Test... current best accuracy is 78.77%
Test... current best accuracy is 79.76%
Test... current best accuracy is 80.24%
Test... current best accuracy is 80.97%
Test... current best accuracy is 81.49%
Test... current best accuracy is 82.35%
Test... current best accuracy is 83.04%
Test... current best accuracy is 83.72%
Test... current best accuracy is 84.26%
Test... current best accuracy is 84.81%
Test... current best accuracy is 85.32%
Test... current best accuracy is 85.71%
Test... current best accuracy is 86.10%
Test... current best accuracy is 86.35%
Test... current best accuracy is 86.42%
Test... current best accuracy is 86.45%
Test... current best accuracy is 86.60%
Best accuracy: 86.60%


## 建構 Variational AutoEncoder 的網路（第四小題）

In [12]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, ctx_size):
        super(VariationalAutoEncoder, self).__init__()
        self.img_size = TARGET_SIZE
        self.ctx_size = ctx_size
        h, w = self.img_size

        self.encoder = nn.Sequential(
            FullyConnect(w * h, 256, nn.SiLU()),
            FullyConnect(256, 128, nn.SiLU()),
            FullyConnect(128, 2 * ctx_size),
        )
        self.decoder = nn.Sequential(
            FullyConnect(ctx_size, 128, nn.SiLU()),
            FullyConnect(128, 256, nn.SiLU()),
            FullyConnect(256, w * h, nn.Sigmoid())
        )
    
    def encode(self, img):
        b, _, _, _ = img.shape
        h, w = self.img_size
        img = torch.reshape(img, (b, h * w))
        x = self.encoder(img)
        mu, logvar = torch.split(x, self.ctx_size, dim=1)
        return self.reparameterize(mu, logvar)

    def decode(self, ctx):
        b, _ = ctx.shape
        h, w = self.img_size
        img = self.decoder(ctx)
        img = torch.reshape(img, (b, 1, h, w))
        return img

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode_with_others(self, img):
        b, _, _, _ = img.shape
        h, w = self.img_size
        img = torch.reshape(img, (b, h * w))
        x = self.encoder(img)
        mu, logvar = torch.split(x, self.ctx_size, dim=1)
        return self.reparameterize(mu, logvar), mu, logvar

    def forward_with_others(self, x):
        input = x
        x, mu, logvar = self.encode_with_others(x)
        x = self.decode(x)
        return x, mu, logvar

    def forward(self, x):
        input = x
        x, mu, logvar = self.encode_with_others(x)
        x = self.decode(x)
        return x

## 訓練 Variational AutoEncoder 的網路（第四小題）

In [13]:
def train_variational_auto_encoder(net, device, loader, loss_type="all"):
    assert loss_type in ["all", "bce", "kld"]
    net = net.to(device)
    net.train()
    bce_loss = nn.BCELoss()
    opt = optim.Adam(net.parameters(),
                     lr=0.002,
                     weight_decay=0.0)

    running_loss = list()

    for e in range(200):
        for imgs, _ in loader:
            imgs = imgs.to(device)
            opt.zero_grad()
            output, mu, logvar = net.forward_with_others(imgs)

            output = torch.clamp(output, min=0.0001, max=0.9999)
            bce = bce_loss(output, imgs)
            kld = torch.mean(-0.5 * torch.sum(1 + logvar - mu ** 2 - logvar.exp(), dim = 1), dim = 0)
            if loss_type == "all":
                loss = bce + kld
            elif loss_type == "bce":
                loss = bce
            elif loss_type == "kld":
                loss = kld
            loss.backward()
            opt.step()

            running_loss.append(loss.item())
            if len(running_loss) > 500:
                running_loss.pop(0)
        if (e+1) % 20 == 0:
            print("epoch {} -> loss: {:.4f}".format(
                      e+1, sum(running_loss)/len(running_loss)))
    return net

vae_net = VariationalAutoEncoder(2)
vae_net = vae_net.to(device)
vae_net = train_variational_auto_encoder(vae_net, device, t_loader, "bce")

epoch 20 -> loss: 0.1373
epoch 40 -> loss: 0.1338
epoch 60 -> loss: 0.1311
epoch 80 -> loss: 0.1301
epoch 100 -> loss: 0.1285
epoch 120 -> loss: 0.1280
epoch 140 -> loss: 0.1272
epoch 160 -> loss: 0.1268
epoch 180 -> loss: 0.1264
epoch 200 -> loss: 0.1255


## 測試 Variational AutoEncoder 的檢測性能（第四小題）

In [14]:
best_vae_net_acc = 0
for i in range(30):
    factor = i/20
    thres = compute_threshold(vae_net, device, t_loader, factor)
    vae_net_acc = compute_accuracy(vae_net, device, a_loader, thres)
    if best_vae_net_acc > vae_net_acc:
        continue
    best_vae_net_acc = vae_net_acc
    print("Test... current best accuracy is {:.2f}%".format(100 * best_vae_net_acc))
print("Best accuracy: {:.2f}%".format(100 * best_vae_net_acc))

Test... current best accuracy is 77.59%
Test... current best accuracy is 79.01%
Test... current best accuracy is 80.27%
Test... current best accuracy is 81.58%
Test... current best accuracy is 83.68%
Test... current best accuracy is 85.03%
Test... current best accuracy is 86.07%
Test... current best accuracy is 86.90%
Test... current best accuracy is 87.82%
Test... current best accuracy is 88.24%
Test... current best accuracy is 88.52%
Test... current best accuracy is 88.64%
Test... current best accuracy is 88.78%
Best accuracy: 88.78%


## 直接使用 Isolated Forest method 做異常檢測（第五小題）

In [15]:
def train_isolation_forest(loader):
    items = torch.zeros(0)
    abnorm_tag = torch.zeros((0), dtype=torch.bool)
    num_abnorms = 0
    for imgs, abnorm in loader:
        items = torch.cat((items, imgs.flatten(start_dim=1)), 0)
        abnorm_tag = torch.cat((abnorm_tag, abnorm), 0)
        num_abnorms += (abnorm == True).sum().item()
    items = items.detach().cpu().numpy()
    abnorm_tag = abnorm_tag.detach().cpu().numpy()
    num_items = items.shape[0]

    if num_abnorms > 0.2 * num_items:
        target_num_abnorms = 0.25 * (num_items - num_abnorms)
        num_abnorms = 0
        new_items = list()
        for c, tag in zip(items, abnorm_tag):
            if tag == True:
                if num_abnorms < target_num_abnorms:
                    new_items.append(c)
                    num_abnorms += 1
            else:
                new_items.append(c)
        items = np.array(new_items)
        num_items = items.shape[0]
    clf = IsolationForest(n_estimators=40,
                          contamination=num_abnorms/num_items,
                          max_samples=round(0.9 * num_items),
                          random_state=99)
    clf.fit(items)
    return clf

def compute_clf_accuracy(loader, clf):
    total = 0
    correct = 0
    for imgs, abnorm in loader:
        imgs = imgs.flatten(start_dim=1).detach().cpu().numpy()
        abnorm = abnorm.detach().cpu().numpy()
        pred = clf.predict(imgs)
        pred = np.where(pred == -1, True, False)

        total += abnorm.shape[0]
        correct += (pred == abnorm).sum()
    return correct/total

clf = train_isolation_forest(r_loader)
best_clf_acc = compute_clf_accuracy(a_loader, clf)
print("Accuracy: {:.2f}%".format(100 * best_clf_acc))

Accuracy: 62.14%


## 使用預訓練的網路配合 Isolated Forest method 做異常檢測（第六小題）

In [16]:
def train_isolation_forest_with_net(net, device, loader):
    net.eval()
    prob_items = torch.zeros(0).to(device)
    abnorm_tag = torch.zeros((0), dtype=torch.bool).to(device)
    with torch.no_grad():
        num_abnorms = 0
        for imgs, abnorm in loader:
            imgs, abnorm = imgs.to(device), abnorm.to(device)
            prob = net.get_prob(imgs)
            prob_items = torch.cat((prob_items, prob), 0)
            abnorm_tag = torch.cat((abnorm_tag, abnorm), 0)
            num_abnorms += (abnorm == True).sum().item()
    prob_items = prob_items.detach().cpu().numpy()
    abnorm_tag = abnorm_tag.detach().cpu().numpy()
    num_items = prob_items.shape[0]

    if num_abnorms > 0.2 * num_items:
        target_num_abnorms = 0.25 * (num_items - num_abnorms)
        num_abnorms = 0
        new_prob_items = list()
        for c, tag in zip(prob_items, abnorm_tag):
            if tag == True:
                if num_abnorms < target_num_abnorms:
                    new_prob_items.append(c)
                    num_abnorms += 1
            else:
                new_prob_items.append(c)
        prob_items = np.array(new_prob_items)
        num_items = prob_items.shape[0]
    clf = IsolationForest(n_estimators=40,
                          contamination=num_abnorms/num_items,
                          max_samples=round(0.9 * num_items),
                          random_state=99)
    clf.fit(prob_items)
    return clf

def compute_clf_accuracy_with_net(net, device, loader, clf):
    with torch.no_grad():
        total = 0
        correct = 0
        for imgs, abnorm in loader:
            imgs = imgs.to(device)
            prob = net.get_prob(imgs)
            prob = prob.detach().cpu().numpy()

            abnorm = abnorm.detach().cpu().numpy()
            pred = clf.predict(prob)
            pred = np.where(pred == -1, True, False)

            total += abnorm.shape[0]
            correct += (pred == abnorm).sum()
    return correct/total
    
clf = train_isolation_forest_with_net(classify_net, device, r_loader)
best_pretrained_clf_acc = compute_clf_accuracy_with_net(classify_net, device, a_loader, clf)
print("Accuracy: {:.2f}%".format(100 * best_pretrained_clf_acc))

Accuracy: 84.61%


## 比較不同方法的最佳結果（第七小題）

In [17]:
print("                                 The classify net (第ㄧ小題) accuracy: {:.2f}%".format(100 * best_classify_net_acc))
print("                                  The AutoEncoder (第二小題) accuracy: {:.2f}%".format(100 * best_ae_net_acc))
print("                        The Denoising Autoencoder (第三小題) accuracy: {:.2f}%".format(100 * best_dae_net_acc))
print("                  The Variational Autoencoder net (第四小題) accuracy: {:.2f}%".format(100 * best_vae_net_acc))
print("                       The Isolated Forest method (第五小題) accuracy: {:.2f}%".format(100 * best_clf_acc))
print("The Isolated Forest method with pre-trained model (第六小題) accuracy: {:.2f}%".format(100 * best_pretrained_clf_acc))

                                 The classify net (第ㄧ小題) accuracy: 88.60%
                                  The AutoEncoder (第二小題) accuracy: 88.04%
                        The Denoising Autoencoder (第三小題) accuracy: 86.60%
                  The Variational Autoencoder net (第四小題) accuracy: 88.78%
                       The Isolated Forest method (第五小題) accuracy: 62.14%
The Isolated Forest method with pre-trained model (第六小題) accuracy: 84.61%


在所有檢測方法中，只要調整好不同方法的閾值，所有使用神經網路輔助的方法，其準確度都差不多，唯有直接使用 Isolated Forest 方法的沒有使用神經網路，它的準確度相對較差。